In [1]:
import os
import sys
import gc
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
# 전체 데이터
with open('./data/RAW_v2_201130.json', 'r') as f:
    raw = json.load(f)

In [3]:
# wine 기본 meta 정보
with open('./data/Wine_Meta_v1_201129.json', 'r') as f:
    wine = json.load(f)

In [4]:
# wine taste(맛) 정보
taste = pd.read_csv('./data/Wine_taste_v1_201129.csv')

In [5]:
# wine food 정보 (meta에 있는 데이터 정리본)
food = pd.read_csv('./data/Wine_food_v2_201207.csv')

In [6]:
# wine review token 정보
with open('./data/Wine_Token_v1_201129.json', 'r') as f:
    token = json.load(f)

In [7]:
df = (pd.DataFrame(zip(
    raw['wine_id'].values(),
    raw['vintage_id'].values(),
    raw['userID'].values(),
    raw['rating_per_user'].values()),
                   columns = ['wine_id', 'vintage_id', 'userID', 'rating'])
     )

In [8]:
df

,wine_id,vintage_id,userID,rating
0,1141133,164942680,19484511,4.0
1,1141133,164942680,352674,4.0
2,1141133,164942680,2148498,1.5
3,1141133,164942680,3450270,4.5
4,1141133,164942680,17786617,4.0
...,...,...,...,...
944094,63654,2435472,11274168,4.0
944095,5602,2293611,11274168,4.5
944096,1396664,8169599,11274168,3.0
944097,1218423,1590767,11274168,4.0


In [9]:
print('wine 개수:', df.wine_id.nunique())

wine 개수: 50861


In [10]:
columns = ['wine_id',
           'name',
           'rating_count',
           'rating_average',
#            'rating_distribution',
           'label_count',
           'review_count',
           'type_id',
           'body', 
           'acidity', 
           'alcohol', 
#            'food', 
           'grapes_id', 
#            'grapes_count', 
#            'grape_composition', 
           'region_id',
           'country_code', 
#            'country_most_used_grapes_id', 
#            'country_most_used_grapes_wines_count', 
           'winery_id', 
           'winery_ratings_count',
           'winery_ratings_average', 
           'winery_labels_count',
           'winery_wines_count']

In [11]:
item = (pd.DataFrame(zip(
    wine['wine_id'].values(), # wine_id
    wine['name'].values(), # wine명
    wine['rating_count'].values(), # wine rating 개수 -> continuous
    wine['rating_average'].values(), # wine rating 평균 -> continuous
#     wine['rating_distribution'].values(), # wine rating
    wine['label_count'].values(), # wine label 개수 -> continuous
    wine['review_count'].values(), # wine review 개수 -> continuous
    wine['type_id'].values(), # wine type -> categorical
    wine['body'].values(), # wine taste - body -> continuous
    wine['acidity'].values(), # wine taste - acidity -> continuous
    wine['alcohol'].values(), # wine taste - alcohol -> continuous
#     wine['food'].values(), # wine with food -> categorical
    wine['grapes_id'].values(), # wine 포도 원산지? -> categorical
#     wine['grapes_count'].values(), # wine 포도 원산지 개수? -> continuous
#     wine['grape_composition'].values(),
#     wine['rank'].values(),
    wine['region_id'].values(), # wine 생산지역 -> categorical
#     wine['region_name'].values(),
    wine['country_code'].values(), # wine 생산국가 -> categorical
#     wine['country_most_used_grapes_id'].values(), wine 생산국가로 구별 가능
#     wine['country_most_used_grapes_wines_count'].values(), wine 생산국가로 구별 가능
    wine['winery_id'].values(), # wine의 winery -> categorical
    wine['winery_ratings_count'].values(),
    wine['winery_ratings_average'].values(),
    wine['winery_labels_count'].values(),
    wine['winery_wines_count'].values())
                    ,columns = columns))

In [12]:
set(df['wine_id']) - set(item['wine_id'])

{1183966, 6298594}

In [13]:
item.loc[item['wine_id'] == 8710264, 'wine_id'] = 6298594
item.loc[item['wine_id'] == 1886805, 'wine_id'] = 1183966

In [14]:
food.loc[food['wine_id'] == 8710264, 'wine_id'] = 6298594
food.loc[food['wine_id'] == 1886805, 'wine_id'] = 1183966

In [15]:
print(set(df['wine_id']) - set(item['wine_id']))
print(set(df['wine_id']) - set(taste['wine_id']))
print(set(df['wine_id']) - set(food['wine_id']))

{1886805}
set()
{1886805}


In [16]:
item = (item
        .merge(food.drop('food', axis = 1), on = 'wine_id', how = 'inner')
        .merge(taste, on = 'wine_id', how = 'inner')
       )
item.head()

,wine_id,name,rating_count,rating_average,label_count,review_count,type_id,body,acidity_x,alcohol,...,dried_fruit_count,dried_fruit_score,dried_fruit_mentions_count,acidity_y,fizziness,intensity,sweetness,tannin,user_structure_count,calculated_structure_count
0,1938520,1882 Cabernet Sauvignon,1697,4.1,14879,16,1,5.0,2.0,14.5,...,6.0,381.0,3.0,3.485010,NaN,4.830508,1.446046,3.571554,81.0,208.0
1,14604,Les Bessards Hermitage,1078,4.3,5370,3,1,5.0,3.0,14.0,...,4.0,287.0,3.0,3.429150,NaN,4.796805,1.703352,4.125626,6.0,83.0
2,1930757,Patriarch Estate Grown,1072,4.6,6042,25,1,4.0,3.0,14.2,...,7.0,771.0,8.0,3.851015,NaN,4.807467,1.517978,3.308112,21.0,127.0
3,1564280,Merlot,3577,4.3,18748,52,1,4.0,3.0,14.4,...,11.0,919.0,9.0,1.775668,NaN,4.012386,1.987188,2.156788,47.0,484.0
4,2576427,Cabernet Sauvignon F Block,115,4.4,806,1,1,5.0,2.0,0.0,...,NaN,NaN,NaN,3.511364,NaN,5.000000,1.272727,3.875000,0.0,7.0


In [22]:
item.shape

(50860, 101)

In [45]:
# token 정리

In [23]:
token.keys()

dict_keys(['red_fruit', 'red_fruit_word', 'citrus_fruit', 'citrus_fruit_word', 'non_oak', 'non_oak_word', 'tree_fruit', 'tree_fruit_word', 'microbio', 'microbio_word', 'earth', 'earth_word', 'oak', 'oak_word', 'black_fruit', 'black_fruit_word', 'vegetal', 'vegetal_word', 'spices', 'spices_word', 'floral', 'floral_word', 'tropical_fruit', 'tropical_fruit_word', 'dried_fruit', 'dried_fruit_word'])

In [24]:
wine_token = []
fail = []
for k in tqdm(token.keys()):
    if 'word' in k:
        continue
    for idx in token[k].keys():
        words = token[k][idx]
        if words == None:
            fail.append(idx)
            continue
        for w in words:
            wine_token.append([idx, w.get('name'), w.get('count')])

100%|██████████| 26/26 [00:03<00:00,  8.08it/s]


In [25]:
wine_token = pd.DataFrame(wine_token, columns = ['wine_id', 'token', 'count'])

In [26]:
wine_token

,wine_id,token,count
0,1141133,strawberry,243
1,1141133,raspberry,169
2,1141133,cherry,76
3,1141133,red fruit,48
4,1141133,cranberry,21
...,...,...,...
2505869,3874005,prune,1
2505870,1196584,dried fruit,2
2505871,1196584,raisin,1
2505872,1196584,dried apricot,1


In [27]:
token_piv = pd.pivot_table(wine_token, index = 'wine_id', columns = 'token', values = 'count').reset_index().fillna(0)

In [28]:
token_piv['wine_id'] = token_piv['wine_id'].astype(int)
token_piv

token,wine_id,5-spice powder,acacia,acai berry,aleppo pepper,allspice,almond,anise,aniseed,apple,...,wood smoke,wood varnish,yeast,yellow apple,yellow beet,yellow peach,yellow plum,yellow raisin,yerba mate,yogurt
0,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100002,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10001,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50768,9998,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50769,99980,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50770,99984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50771,99986,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
# 해당 wine_id (88개)는 review token이 발견되지 않는것으로 생각됨
set(df['wine_id']) - set(token_piv['wine_id'])

{81983,
 91092,
 1164358,
 1175295,
 1220013,
 1245176,
 1255256,
 1259869,
 1269866,
 1277237,
 1283772,
 1289868,
 1318278,
 1469868,
 1481743,
 1488599,
 1499544,
 1518037,
 1524150,
 1556351,
 1577300,
 1624143,
 1639595,
 1665923,
 1666199,
 1690453,
 1817063,
 1819735,
 1837570,
 1852097,
 1858809,
 1969047,
 1972533,
 2030825,
 2109027,
 2179598,
 2294826,
 2365048,
 2391977,
 2461053,
 2495655,
 2543676,
 2592516,
 2595710,
 2698232,
 2817325,
 2850571,
 2954728,
 3169411,
 3185584,
 3566663,
 3835115,
 4007118,
 4160566,
 4162368,
 4460813,
 4629941,
 4919079,
 4920014,
 4986250,
 5005763,
 5095343,
 5193687,
 5215664,
 5251616,
 5455491,
 5475945,
 5494648,
 5606226,
 5706917,
 5724753,
 5907868,
 5935915,
 5959679,
 5974169,
 6004332,
 6052553,
 6107611,
 6213819,
 6505872,
 6703532,
 6801749,
 6803602,
 6966533,
 7214287,
 7337460,
 7526357,
 7957468}

In [30]:
none_token = set(df['wine_id']) - set(token_piv['wine_id'])
none_token = pd.DataFrame(none_token, columns = ['wine_id'])
none_token

,wine_id
0,1837570
1,2179598
2,1481743
3,5251616
4,2294826
...,...
83,1817063
84,2954728
85,7337460
86,2698232


In [31]:
token_piv = pd.concat([token_piv, none_token], axis = 0).reset_index(drop = True)

In [32]:
token_piv['wine_id'] = token_piv['wine_id'].astype(int)

In [35]:
token_piv = token_piv.fillna(0)
token_piv.index = token_piv['wine_id']
token_piv = token_piv.drop('wine_id', axis = 1)
token_piv

,5-spice powder,acacia,acai berry,aleppo pepper,allspice,almond,anise,aniseed,apple,apple blossom,...,wood smoke,wood varnish,yeast,yellow apple,yellow beet,yellow peach,yellow plum,yellow raisin,yerba mate,yogurt
wine_id,,,,,,,,,,,,,,,,,,,,,
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100002,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100006,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10001,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1817063,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2954728,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7337460,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
# 너무 차원이 크기 때문에 차원축소 ㄱㄱ
from sklearn.decomposition import PCA
pca = PCA(0.99)

In [41]:
token_pca = pd.DataFrame(pca.fit_transform(token_piv), columns = ['pca_{}'.format(i) for i in range(1, 52)], index = token_piv.index)

In [43]:
token_pca = token_pca.reset_index()
token_pca

,wine_id,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,...,pca_42,pca_43,pca_44,pca_45,pca_46,pca_47,pca_48,pca_49,pca_50,pca_51
0,10,-34.816310,-10.297108,4.113811,1.313828,-0.402693,-1.011831,-1.751832,-1.175891,2.529086,...,-0.348280,0.052000,1.002579,0.587195,0.464686,-0.756110,-0.261854,0.319835,-0.936683,-0.367839
1,10000,-16.598365,-12.336662,3.401326,4.088621,4.086811,3.824394,1.204167,-4.070763,1.308612,...,2.574157,0.297820,0.606474,-0.373117,0.640388,0.552938,0.523138,-1.347733,0.845015,1.302563
2,100002,8.613945,-16.817992,-3.691811,-6.873882,-4.621727,-2.045158,-6.960172,5.698033,-6.714681,...,-1.239114,-0.059552,1.500062,0.133128,2.083096,0.676100,-2.070532,4.080133,3.092238,-0.003180
3,100006,-16.863190,-12.686163,5.977805,-1.170474,-5.372541,-0.315191,-0.548081,-1.284229,-0.814350,...,1.343874,-0.877989,-1.008183,0.695315,0.389120,-0.047576,0.398050,-0.536036,-0.525845,-0.018300
4,10001,-25.822374,-1.760176,16.175796,-2.059567,-0.914461,-0.630260,4.081136,-1.604363,1.553382,...,-0.040564,-0.146263,-0.717619,-1.430608,-0.197574,-1.025089,-1.726425,-0.661659,0.217338,0.480206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50856,1817063,-37.238381,-12.217772,5.827138,-2.547189,-2.064486,0.668959,-0.166074,-2.529731,1.631889,...,0.022690,-0.013990,-0.040600,0.034953,-0.103948,-0.108181,0.084244,0.113221,-0.047842,-0.053844
50857,2954728,-37.238381,-12.217772,5.827138,-2.547189,-2.064486,0.668959,-0.166074,-2.529731,1.631889,...,0.022690,-0.013990,-0.040600,0.034953,-0.103948,-0.108181,0.084244,0.113221,-0.047842,-0.053844
50858,7337460,-37.238381,-12.217772,5.827138,-2.547189,-2.064486,0.668959,-0.166074,-2.529731,1.631889,...,0.022690,-0.013990,-0.040600,0.034953,-0.103948,-0.108181,0.084244,0.113221,-0.047842,-0.053844
50859,2698232,-37.238381,-12.217772,5.827138,-2.547189,-2.064486,0.668959,-0.166074,-2.529731,1.631889,...,0.022690,-0.013990,-0.040600,0.034953,-0.103948,-0.108181,0.084244,0.113221,-0.047842,-0.053844


In [44]:
# item = item.merge(token_piv, on = 'wine_id', how = 'inner')
item = item.merge(token_pca, on = 'wine_id', how = 'inner')

In [45]:
# 해당 acidity는 meta와 taste에서 온걸로 판단됨
# acidity_x는 meta에서
# acidity_y는 taste에서..
item[item.columns[item.columns.str.contains('_x|_y')]]

,acidity_x,acidity_y
0,2.0,3.485010
1,3.0,3.429150
2,3.0,3.851015
3,3.0,1.775668
4,2.0,3.511364
...,...,...
50855,3.0,3.212859
50856,3.0,3.072673
50857,3.0,3.982507
50858,3.0,4.061171


In [46]:
# item.to_csv('./data/Wine_Meta_final_201208.csv', index = False)
item.to_csv('./data/Wine_Meta_final_201229.csv', index = False)

In [47]:
item

,wine_id,name,rating_count,rating_average,label_count,review_count,type_id,body,acidity_x,alcohol,...,pca_42,pca_43,pca_44,pca_45,pca_46,pca_47,pca_48,pca_49,pca_50,pca_51
0,1938520,1882 Cabernet Sauvignon,1697,4.1,14879,16,1,5.0,2.0,14.5,...,-3.136951,0.742006,0.173241,-1.924884,-1.610956,2.868221,-2.167123,1.151749,1.444787,2.489641
1,14604,Les Bessards Hermitage,1078,4.3,5370,3,1,5.0,3.0,14.0,...,-1.082327,0.338037,-2.199833,-0.638129,1.981586,1.148229,-0.780446,-1.026985,-3.631833,-0.124608
2,1930757,Patriarch Estate Grown,1072,4.6,6042,25,1,4.0,3.0,14.2,...,-0.657080,0.566004,0.658332,-0.343338,-1.285816,0.543290,-0.569400,1.647680,-1.445715,-0.359417
3,1564280,Merlot,3577,4.3,18748,52,1,4.0,3.0,14.4,...,2.592272,-0.271975,1.884531,0.176404,4.579372,1.793655,-4.051642,3.926317,-2.261881,-0.431446
4,2576427,Cabernet Sauvignon F Block,115,4.4,806,1,1,5.0,2.0,0.0,...,-0.016644,-0.063100,-0.098678,0.082621,0.164888,-0.078193,0.113466,0.270745,0.158934,-0.330067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50855,1669561,Garganega - Pinot Grigio,788,3.5,6635,9,2,3.0,3.0,0.0,...,0.081186,0.734952,-0.616824,0.442448,-0.520157,2.901341,-2.457316,-2.278633,-0.337037,-2.108658
50856,1861275,Dadà Langhe Chardonnay,231,3.8,961,6,2,3.0,3.0,13.5,...,0.031847,-0.799679,0.032218,0.221882,-0.216117,1.380378,0.212103,-0.010373,-0.298942,0.240547
50857,2201892,Metodo Zero Prosecco Extra Dry,390,3.9,1983,14,3,1.0,3.0,0.0,...,-0.053284,-0.092606,0.169206,0.914804,-0.225595,0.838082,-0.267864,-0.312077,-0.135752,-0.265988
50858,2396179,Les Monts Damnés Sancerre,302,4.2,730,4,2,4.0,3.0,13.5,...,0.099739,-1.552876,-0.237777,0.776565,0.619847,-2.632533,-2.494865,-0.542009,0.079459,-0.041871
